In [1]:
import os
import pyspark
conf = pyspark.SparkConf()

# conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True) # enabled for debuggig 
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

23/12/12 17:08:18 WARN Utils: Your hostname, Pragnavis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.53 instead (on interface en0)
23/12/12 17:08:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 17:08:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/pragnavi/anaconda3/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [31]:
#from pyspark.sql.functions import col , column, sum, dayofweek, udf, count, rank, collect_list, concat_ws, concat, lit, split, struct, trim, explode, lower, regexp_replace
# from pyspark.sql.types import IntegerType, DoubleType
# from pyspark.sql.window import Window
# from pyspark.ml.feature import NGram

from pyspark.sql.functions import col, to_date
from pyspark.sql.types import DateType, StructType, StructField, DoubleType, StringType
from itertools import product
from pyspark.sql import functions as F

In [ ]:
#ARREST_DATE, ARREST_BORO, AGE_GROUP, PERP_SEX, PERP_RACE, Latitude, Longitude

In [18]:
arrest_data_df = spark\
.read.option("header",True)\
.option("inferSchema",True)\
.csv("data")

arrest_data_df.printSchema()

root
 |-- ARREST_KEY: string (nullable = true)
 |-- ARREST_DATE: string (nullable = true)
 |-- PD_CD: string (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: string (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: string (nullable = true)
 |-- JURISDICTION_CODE: string (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- X_COORD_CD: string (nullable = true)
 |-- Y_COORD_CD: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Lon_Lat: string (nullable = true)



In [19]:
arrest_data_df = arrest_data_df.withColumn("ARREST_DATE", to_date(arrest_data_df["ARREST_DATE"], "MM/dd/yyyy").cast(DateType()))
arrest_data_df = arrest_data_df \
    .select(col("ARREST_DATE"), col("ARREST_BORO"), col("AGE_GROUP"), col("PERP_SEX"), col("PERP_RACE"), col("Latitude"), col("Longitude")) 

In [20]:
arrest_data_df.orderBy(col("ARREST_DATE").desc())

ARREST_DATE,ARREST_BORO,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude
2023-09-30,M,25-44,M,WHITE,40.7623963379051,-73.9786009241232
2023-09-30,K,25-44,M,WHITE HISPANIC,40.641462,-73.980247
2023-09-30,M,25-44,M,BLACK,40.768890992016146,-73.98209349413949
2023-09-30,Q,45-64,M,WHITE HISPANIC,40.7927597502013,-73.84842984775335
2023-09-30,Q,45-64,F,BLACK,40.6799807384666,-73.7762339071953
2023-09-30,K,45-64,M,BLACK,40.57604135964683,-73.99007546483202
2023-09-30,K,45-64,M,BLACK,40.5993145044713,-73.97290957437446
2023-09-30,B,25-44,M,BLACK,40.853578,-73.900591
2023-09-30,Q,45-64,M,WHITE,40.79152936748136,-73.84175152958223
2023-09-30,M,18-24,M,BLACK HISPANIC,40.758679,-73.99516


In [42]:
#Zone Division

# Define the range of coordinates for New York City
min_latitude, max_latitude = 40.7, 40.901
min_longitude, max_longitude = -74.05, -73.899

# Define the grid step (1 mile difference)
step = 0.0005 # Approximately 1 mile in degrees

# Generate latitude and longitude values using Python range function
latitude_values = [round(lat, 6) for lat in
                   range(int(min_latitude * 1000000), int(max_latitude * 1000000), int(step * 1000000))]
latitude_values = [lat / 1000000 for lat in latitude_values]

longitude_values = [round(lon, 6) for lon in
                    range(int(min_longitude * 1000000), int(max_longitude * 1000000), int(step * 1000000))]
longitude_values = [lon / 1000000 for lon in longitude_values]

# Create a list of combinations of longitude and latitude values
combinations = list(product(longitude_values, latitude_values))

# Create a schema for the grid DataFrame
schema = StructType([
    StructField("x1-coordinate", DoubleType(), True),
    StructField("y1-coordinate", DoubleType(), True),
    StructField("x2-coordinate", DoubleType(), True),
    StructField("y2-coordinate", DoubleType(), True),
    StructField("zone-id", StringType(), True)
])

# Create a list of structured data tuples for the DataFrame
grid_data = [(x1, y1, x2, y2, f'{x1}_{y1}') for (x1, y1), (x2, y2) in zip(combinations[:-1], combinations[1:])]

# Create a DataFrame from the data and schema
grid_df = spark.createDataFrame(grid_data, schema=schema)

# Display the grid DataFrame
grid_df

x1-coordinate,y1-coordinate,x2-coordinate,y2-coordinate,zone-id
-74.05,40.7,-74.05,40.7005,-74.05_40.7
-74.05,40.7005,-74.05,40.701,-74.05_40.7005
-74.05,40.701,-74.05,40.7015,-74.05_40.701
-74.05,40.7015,-74.05,40.702,-74.05_40.7015
-74.05,40.702,-74.05,40.7025,-74.05_40.702
-74.05,40.7025,-74.05,40.703,-74.05_40.7025
-74.05,40.703,-74.05,40.7035,-74.05_40.703
-74.05,40.7035,-74.05,40.704,-74.05_40.7035
-74.05,40.704,-74.05,40.7045,-74.05_40.704
-74.05,40.7045,-74.05,40.705,-74.05_40.7045
